In [ ]:
import pymongo
import pandas as pd
import json



from pymongo.mongo_client import MongoClient
from pprint import pprint

In [ ]:
with open('config.json') as f:
    config = json.load(f)
uri = config.get("MONGO_URI")
client = MongoClient(uri)

# Test Connection
try:
    client.admin.command('ping')
    print("pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


#Creating a DB

db = client["Mav"]
sales = db["sales"]


In [ ]:
# Getting random generated info sales
df = pd.read_csv('random.csv')

# Transforming data types and lowering case
df['total_price'] = df.total_price.str.replace('€','')
df['total_price'] = df.total_price.str.replace(',','.')
df['total_price'] = df.total_price.astype(float)

df = df.map(lambda x: x.lower() if isinstance(x,str) else x)

# Transforming to a dict for easy insertion to the collection in MongoDB 
datos_dict = df.to_dict(orient='records')

# Inserting the dict to the mongoDB
for doc in datos_dict:
    sales.insert_one(doc)

In [ ]:
#Adding the document "Year"
import random

year_range = list(range(2000,2024))

for document in sales.find():
    random_year = random.choice(year_range)
    sales.update_one({"_id":document["_id"]}, {"$set":{"year":random_year}})


#Checking if the records got inserted

pprint(sales.find_one())



#Delete the year field
# sales.update_many({},{"$unset":{"year":1}},True)